In [1]:
import pandas as pd
import data

filepath = "../data/papers/urban-agriculture.csv"
df = pd.read_csv(filepath)
df = data.load_scopus(df)

2020-06-06 01:30:46,094 - INFO - Renaming and selecting columns ...
2020-06-06 01:30:51,669 - INFO - Formatting author names ...
2020-06-06 01:30:51,674 - INFO - Disambiguating author names ...
2020-06-06 01:30:51,738 - INFO - Removing part of titles in foreing languages ...
2020-06-06 01:30:51,741 - INFO - Fusioning author and index keywords ...
2020-06-06 01:30:51,753 - INFO - NumExpr defaulting to 8 threads.
2020-06-06 01:30:51,767 - INFO - Extracting countries from affiliations ...
2020-06-06 01:30:57,883 - INFO - Extracting institutions from affiliations ...
2020-06-06 01:30:57,893 - INFO - Extracting country of 1st author ...
2020-06-06 01:30:57,896 - INFO - Extracting affiliation of 1st author ...
2020-06-06 01:30:57,900 - INFO - Counting number of authors ...


### Basic info

In [ ]:
data.coverage(df)

### Keywords

### FindString Dialog Box pag. 69

### Cleaning a list pag. 102

### Fuzzy match editor pag.126

### Stemming And / Stemming Or

### APPS

In [ ]:
import by_year as year
year.app(df)

In [ ]:
import by_term as term
term.app(df)

In [ ]:
import by_term_per_year as term_year
term_year.app(df)

In [2]:
import co_occurrence as co_occurrence
co_occurrence.app(df)

AppLayout(children=(HTML(value="<h1>Co-occurrence Analysis</h1><hr style='height:2px;border-width:0;color:gray…

In [ ]:
import correlation as correlation
correlation.app(df)

In [ ]:
import factor_analysis

factor_analysis.app(df)